Este desarrollo pretende comparar los precios de una cesta de la compra en dos supermercados. Tambien ppodría usarse, previo formación de un historico y una pequeña modificación,  para estudiar la varación del IPC de dicha cesta.

Para ello se creua lista con productos con un nombre estandar acorde con las denominaciones de cada supermercado con objeto que al buscar sea el primer elemento del resultado de la busqueda.

Se ha tomado com referencia Mercadona y Dia, y una cesta de cinco productos.

EL proceso de busqueda consiste en entrar en la pagina web, saltarse la cookies e ir buscando cada producto de la lista, inicalizando previamente el campo de la busqueda de la pagina web.

Se obtiene una lista valorada por cada supermercado, fundiendose en una sola donde se compara precios y se recomeinda el supermercado donde realziar la compra.
Al final produce un fichero csv con la decisión de donde comprar cada producto en función del precio más barato.
Tambien se obtiene la descripcion y formato de un producto por cada supermercado.
A efectpos didacticos se inccluye una clasificación de cada supermercado, de acuerdo con la clasificación del B15_OCT_Grandesuperficies.pdf, apra ello se lee la pagina, se busca los supermercados en el grupo en el que están.

In [1]:
#!pip install PyPDF2

In [2]:
# !pip install tabula-py

In [3]:
# !pip install beautifulsoup4

In [4]:
# !pip install webdriver-manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [5]:
# !pip install pymupdf

In [6]:
import requests as req
import pandas as pd
import fitz
import time  # manejo temporal

from bs4 import BeautifulSoup as bs

from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent

PATH = r'C:\Users\ragod\OneDrive\Documentos\GitHub\ds_tb_part_21_09\02-Data_Analysis\driver\chromedriver.exe'  #'driver/chromedriver'
usuario=UserAgent().random


from selenium import webdriver  # driver selenium
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By   # para elegir por elemento
from selenium.webdriver.support.ui import WebDriverWait # para el time sleep
from selenium.webdriver.support import expected_conditions as EC # condiciones esperadas
from selenium.webdriver import ActionChains as AC # acciones en cadena


In [7]:
def clasificador(empresa):
    if empresa == 'Mercadona':
        clasif = clasi_Mercadona
    else:
        clasif = clasi_Dia
    return clasif

In [8]:
# Busqueda de la calsificacion en un archivo pdf
doc = fitz.open(r'C:/Users/ragod/OneDrive/Escritorio/B15_OCT_Grandesuperficies.pdf') 
#lectura d ela pagina de clasificaciones
pagina = doc[14]
paras = pagina.get_text('text')
#pagina por parrafos
parrafos = paras.split('\n \n')
#Busqueda de clasificaciones, primer palabra de parrafos 8 y 9
#Clasificaciones
clase1 = parrafos[8][3:15]
clase2 = parrafos[9][3:42]
clasificacion ={8:clase1,9:clase2}
#Busqueda de clasificación de Dia y mercadona
for n in range(len(parrafos)):
    if parrafos[n].find('Mercadona') > 0 : clasi_Mercadona = clasificacion.get(n)
    if parrafos[n].find('Día') > 0 : clasi_Dia = clasificacion.get(n)

In [9]:
opciones=Options()

opciones.add_argument('--start-maximized')  # empieza maximizado

opciones.add_argument('--headlees')
opciones.headless=False               # Si es True no aparece la pantalla

opciones.add_experimental_option('excludeSwitches', ['enable-automation']) # para quitar flag de robot
opciones.add_experimental_option('useAutomationExtension', False)

#opciones.add_extension('driver/adblock.crx')  # extension de adblocker

opciones.add_argument('--incognito')

opciones.add_argument('user-data-dir=selenium')   # para guardar cookies

opciones.add_argument('--no-proxy-server')  # sin proxy
opciones.add_argument('proxy-server=106.122.8.54:3128')  # proxy custom
# usuario=UserAgent().random opciones del driver

In [10]:
url='https://tienda.mercadona.es/'
driver=webdriver.Chrome(PATH) 
driver.get(url)

time.sleep(3)


<ipython-input-10-89b6f3082186>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)



In [11]:

#Cesta de la compra

mi_lista =['HUEVO GALLINA CAMPERA','CALABAZA EN TROZOS', 'KEFIR','ESPARRAGO VERDE','UVA BLANCA SIN SEMILLA']

#Busqueda en Mercadona
saltar_coockies=driver.find_element(By.XPATH,'//*[@id="root"]/div[1]/div/div/button[2]')
saltar_coockies.click()
# introducir codigo postal
busqueda = '28016'
buscar=driver.find_element(By.XPATH,'//*[@id="root"]/div[4]/div/div[2]/div/form/div/input') # objeto en la web de busqueda

buscar.send_keys(busqueda)  # incrusta la string
time.sleep(4)


primer_aceptar=driver.find_element(By.XPATH,'//*[@id="root"]/div[4]/div/div[2]/div/form/button')
primer_aceptar.click()

mi_lista_valorada =[]


#buscar productos
for n in range(len(mi_lista)):
    buscar=driver.find_element(By.XPATH,'//*[@id="search"]').clear()
    producto = mi_lista[n]
    buscar=driver.find_element(By.XPATH,'//*[@id="search"]')
    buscar.send_keys(producto)  # incrusta la string
    time.sleep(3)
    producto_mercadona = driver.find_element_by_class_name('product-container').text
    producto_mercadona_list=producto_mercadona.split('\n')
    producto_mercadona_descripcion = producto_mercadona_list[0]
    producto_mercadona_formato = producto_mercadona_list[1]
    producto_mercadona_precio = producto_mercadona_list[2].replace(' € /ud.','')
    # Crear lista valorada Mercadona
    mi_lista_valorada.append([producto, producto_mercadona_precio])




<ipython-input-11-2f44200a677f>:29: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  producto_mercadona = driver.find_element_by_class_name('product-container').text



In [12]:
# Busqueda en Dia
url_dia ='https://www.dia.es/compra-online/'
driver=webdriver.Chrome(PATH)
driver.get(url_dia)

time.sleep(4) # La cookie tarda muchho en salir, por eso se establece a 4.
# saltar cookies
saltar_coockies=driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]')
saltar_coockies.click()
mi_lista_valorada2=[]
#buscar productos
for n in range(len(mi_lista)):
    buscar=driver.find_element(By.XPATH,'//*[@id="search"]').clear()
    producto = mi_lista[n]
    buscar=driver.find_element(By.XPATH,'//*[@id="search"]')
    buscar.send_keys(producto)  # incrusta la string
    time.sleep(3)
    
    producto_Dia_precio = driver.find_element_by_class_name('price').text
    producto_Dia_detalle =driver.find_element_by_class_name('name').text
    #Relleno  de la lista valorada de Dia
    mi_lista_valorada2.append([producto,producto_Dia_precio.replace('€','').strip()])
  

 

<ipython-input-12-9fda7a836a25>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)

<ipython-input-12-9fda7a836a25>:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  producto_Dia_precio = driver.find_element_by_class_name('price').text

<ipython-input-12-9fda7a836a25>:20: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  producto_Dia_detalle =driver.find_element_by_class_name('name').text



In [13]:
df2 =pd.DataFrame(mi_lista_valorada2, columns =['Producto','Precio_Dia'])
#df2['Formato_Dia']=df2.apply(lambda x: x['Denominación_Dia'][len(x['Denominación_Dia'])-6:])
df1= pd.DataFrame(mi_lista_valorada, columns =['Producto','Precio_Mercadona'])

df2['Precio_Mercadona'] =df1['Precio_Mercadona']

df2['Precio_Mercadona'] = df2['Precio_Mercadona'].replace(",",".", regex = True).astype(float)
df2['Precio_Dia'] =df2['Precio_Dia'].replace(",",".", regex = True).astype(float)




ValueError: could not convert string to float: ''

In [ ]:
#Selecciona donde comprar
df2['Donde_comprar']=df2.apply(lambda x: 'Mercadona' if x.Precio_Dia > x.Precio_Mercadona else 'Dia' ,axis=1)

In [ ]:
# Añadir clasificación
df2['Tipo_Clasificacion'] = df2.apply(lambda x: clasificador(x[x.index[3]]),axis =1)


In [ ]:

# Crea CSV
df2.to_csv('compra.csv', index = False)


In [ ]:
df2
